## Honors Project Report: Distrubuted Facial Expression Recognition
#### Samuel Zeleke & Enoch Mwesigwa


### Vision
Multi-label classification and object detection models have gained significant popularity in recent years. They've become
integral components of systems ranging from autonomous cars and security systems, to social media platforms and search
engines. Our project is aimed at creating a system that correctly labels facial expressions in real-time on raspberry pi server
using a CNN built on keras. Our system should be responsive enough to recognize facial expressions in video streams and light
enough to be hosted on a raspberry pi.

### Background
Object detection involves recognizing multiple objects at different position in the image. Unfortunately, regular
CNNs cannot (at least not "normally") solve this problem: their architecture only allows inputs and outputs
with fixed sizes. So, they are restricted to merely labelling images.

There have been several attempts to go around this problem. The most significant ones are using R-CNN and YOLO. [R-CNN](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e)
and its decedents use additional pre-processing to generate thousands of candidate regions (a.k.a. proposals) in an image
and pass each region to a CNN for classification. Obviously, this is a very resource-intensive process, (training in
some architectures takes days) and it's not fast enough for real-time object recognition. [YOLO](https://medium.com/analytics-vidhya/yolo-v3-theory-explained-33100f6d193), on the other hand, uses a single (very) deep CNN to
both recognize regions of interest and to classify those regions. This method is several times faster and more
efficient than R-CNNs. Unfortunately, the architecture needs a lot of data for training and uses NN layers we were not familiar with.

So, building on Gurav Sharma's article "[Real Time Facial Recognition](https:/medium.com/datadriveninvestor/real-time-facial-expression-recognition-f860dacfeb6a)",
we chose to create a simpler system that combines openCV's trained cascade classifiers to extract the
faces and trained a small NN to classify the facial expressions. This largely avoids R-CNNs inefficiencies and significantly reduces the
size of training data we need to get decent predictions.

### Implementation

#### Training
Like R-CNNs, our system divides the facial expression task into two stages. In the first stage, we use openCV's pre-trained cascade classifiers
to find regions containing faces. For the second stage, Sharma recommends taking advantage of the Keras' pretrained models using transfer learning.
(Transfer learning involves using layers from a model trained for a different dataset. It let's the recepient model to take advantage of the "donor"
model's training by using its weights for predictions.) Unfortunately, we didn't have enough training data to achieve significant training set accuracy.
Additionally, the resulting models were taking a lot of storage. Instead, we adopted the architecture we used for the fashion mnist homework to build a small
CNN that classified facial expressions.

*Code for second-stage model training*

In [ ]:
def train_model(batch_size = 10):
    import os
    import zipfile
    import cv2
    import tensorflow as tf
    import keras
    from keras import layers
    # import matplotlib.pyplot as plt
    import numpy

    # unzipfile
    with zipfile.ZipFile("/content/drive/My Drive/tif_extended.zip", 'r') as zip_ref:
        zip_ref.extractall("./trainingSrc")

    PATH = "./trainingSrc/tif_extended"
    CLASSES = os.listdir(PATH)

    image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split = 0.2)
    images = image_generator.flow_from_directory(
        PATH,
        batch_size = 1,
        target_size = (256, 256),
    )
    sample, label = next(images)
    feature_dataframe = []
    target_dataframe = []

    for i in range(len(images)):
      feature_dataframe.append(images[i][0][0])
      target_dataframe.append(images[i][1][0])

    feature_dataframe = numpy.array(feature_dataframe)
    target_dataframe = numpy.array(target_dataframe)

    # create network
    model = keras.Sequential()

    # input and first convolution: extract 30 features
    model.add(keras.layers.Conv2D(30, 2, activation="relu", input_shape = (256, 256, 3)))
    model.add(keras.layers.Conv2D(60, kernel_size=5, strides=(2, 2), activation="relu"))#(60, 5, stri activation="relu"))
    # input and second convolution: extract 30 features
    model.add(keras.layers.Conv2D(60, 5, activation="relu"))
    model.add(keras.layers.Conv2D(30, 3, activation="relu"))
    model.add(keras.layers.MaxPooling2D(2))

    # input and third convolution: extract 30 features
    model.add(keras.layers.Conv2D(60, 5, activation="relu"))
    model.add(keras.layers.MaxPooling2D(2))

    #flatten
    model.add(keras.layers.Flatten())
    # three dense layers
    model.add(keras.layers.Dense(120, activation="relu"))
    model.add(keras.layers.Dense(28, activation="relu"))
    model.add(keras.layers.Dense(7, activation="softmax"))

    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=["acc"]
    )

    model.summary()

    model.fit(
    x = feature_dataframe,
    y = target_dataframe,
    batch_size = batch_size,
    epochs = 10,
    validation_split = 0.2
    )

    return model

Finding useful training data for our project was a challenging process. Most face detection datasets are made to train binary classifiers that
detect whether there is a face in an image or not. Additionally, the larger datasets mentioned in Sharma's article were neither available nor improperly labelled. So,
we used the following code to scrape images from Google Image search. Our scrapper use's selenium's webdriver to grab the images
from the site and uses the cascade classifier to find regions that contain faces and generate new images for those regions.
It then saves the images into directories created using the search term.

*Code for scrapper*

In [ ]:
import os
import time
import selenium
from selenium import webdriver
import cv2 as cv
import requests
import numpy as np
from PIL import Image

DRIVER_PATH = "./chromedriver"

class FaceScraper:

    def __init__(self, path_to_driver=DRIVER_PATH, path_to_face_model="./face_detector.xml", path_to_eye_model="./eye_detector.xml"):

        self._img_urls = dict()
        self._images = dict()
        self._wdriver_path = DRIVER_PATH
        self._face_cascade = cv.CascadeClassifier()
        self._eye_cascade = cv.CascadeClassifier()

        #-- 1. Load the cascades
        if not self._face_cascade.load(path_to_face_model):
            print('--(!)Error loading face cascade')
            exit(0)
        if not self._eye_cascade.load(path_to_eye_model):
            print('--(!)Error loading eye cascade')
            exit(0)

    def getImgUrls(self, search_terms=["smiling", "sad", "surprised", "angry", "neutral", "disgust"], max_num_links = 100, ):
        if (search_terms != None):
            self._search_terms = search_terms

        wd = webdriver.Chrome(executable_path = DRIVER_PATH)

        for term in self._search_terms:
            self._img_urls[term] = self._fetch_image_urls(term, wd, max_links_to_fetch=max_num_links, sleep_between_interactions=0.1)

        wd.quit()

    def extractFaces(self):
        if len(self._img_urls.keys()) == 0:
            raise ValueError("No images in object.")

        for label in self._img_urls.keys():
            results = []
            print("Extracting label: %s\n" % label)

            i = -1
            for url in self._img_urls[label]:
                try:
                    i += 1
                    print("  Extracting label: {} no: {}; url: {}".format(label, i, url))
                    resp = requests.get(url, stream=True).raw
                    print("\tGrabbed image from server")
                    image = np.asarray(bytearray(resp.read()), dtype="uint8")
                    print("\tConverted to an array")
                    image = cv.imdecode(image, cv.IMREAD_COLOR)
                    print("\tDecoded image")

                    print("\tGetting faces")
                    for face in self._detectFace(image):
                        results.append(face)
                except:
                    print("    error: couldn't extract faces for url")
                    continue

            self._images[label] = results

    def saveCropped(self, parent_dir=os.getcwd(), image_type = "png"):
        if len(self._images.keys()) == 0:
            raise ValueError("No images in object.")

        path_to_srcapped = parent_dir + "/scrapped_images"
        i = 1
        while (os.path.exists(path_to_srcapped)):
            path_to_srcapped = path_to_srcapped + "_" + str(i)
            i += 1
        print ("Saving to %s" % path_to_srcapped)
        os.mkdir(path_to_srcapped)
        for label in self._images:
            try:
                print("Exteracting for %s" % label)
                labelDir = path_to_srcapped + "/" + label
                os.mkdir(labelDir)
            except OSError:
                print("Unable to write images under %s label\n" % label)
                continue

            for index in range(len(self._images[label])):
                image_path =  labelDir + "/" + label + "_" + str(index) + "." + image_type
                cv.imwrite(image_path, self._images[label][index])
                try:
                    print("Progress: %.2f%" % 100 * index/len(self._images[label]))
                except:
                    continue
    def _detectFace(self, frame):
        print("\t  preprocessing image", end="... ")
        frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        frame_gray = cv.equalizeHist(frame_gray)
        #-- Detect faces and eyes
        print("Detecting faces image", end="... ")
        faces = self._face_cascade.detectMultiScale(frame_gray)
        eyes = self._eye_cascade.detectMultiScale(frame_gray)
        print("veryfiying", end="... ")
        real_faces = []
        for (x, y, w, h) in faces:
            # x_min, y_min, x_max, y_max = x, y, x + w, y + h
            # for (x_eye, y_eye, w_eye, h_eye) in eyes:
            #     if (x_min <= x_eye and x_eye <= x_max) and (y_min <= y_eye and y_eye <= y_max):
            #         real_faces.append((x, y, w, h))
            #         break
            real_faces.append((x, y, w, h))
            if len(real_faces) > 5:
                break
        print("\t  Done!")
        return [frame[y:y+h,x:x+w] for (x,y,w,h) in real_faces]

    # source
    # https://towardsdatascience.com/image-scraping-with-python-a96feda8af2d
    def _fetch_image_urls(self, query, wd, max_links_to_fetch, sleep_between_interactions=1):
        def scroll_to_end(wd):
            wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)

        # build the google query
        search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

        # load the page
        wd.get(search_url.format(q=query))

        image_urls = set()
        image_count = 0
        results_start = 0
        while image_count < max_links_to_fetch:
            scroll_to_end(wd)

            # get all image thumbnail results
            thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
            number_results = len(thumbnail_results)

            print("Found: {0} search results. Extracting links from {1}:{0}".format(number_results, results_start))

            for img in thumbnail_results[results_start:number_results]:
                # try to click every thumbnail such that we can get the real image behind it
                try:
                    img.click()
                    time.sleep(sleep_between_interactions)
                except Exception:
                    continue

                # extract image urls
                actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
                for actual_image in actual_images:
                    if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                        image_urls.add(actual_image.get_attribute('src'))

                image_count = len(image_urls)

                if len(image_urls) >= max_links_to_fetch:
                    print("Found: {} image links, done!".format(len(image_urls)))
                    break
            else:
                print("Found:", len(image_urls), "image links, looking for more ...")
                time.sleep(15)
                #return image_urls
                load_more_button = wd.find_element_by_css_selector(".mye4qd")
                if load_more_button:
                    wd.execute_script("document.querySelector('.mye4qd').click();")
                else:
                    return image_urls


            # move the result startpoint further down
            results_start = len(thumbnail_results)

        return image_urls



def run():
    list_of_search_terms = [
        "people at weddings",
        "depression human face",
        "people at senate hearing",
        "disgusted face",
        "people shocked"
        ]

    getFaces = FaceScraper()
    getFaces.getImgUrls(list_of_search_terms, 200)
    getFaces.extractFaces()
    getFaces.saveCropped()

# Run scrapper
#run()

This process increased or training data 3-fold. However, because the images were grabbed from a search engine, they also introduced some unintended bias.

#### Recognition
Our project's original aim was to deploy the model on a server hosted on a raspberry pi that gets video feeds from a
client (a headset). However, to meet the pi's hardware performance restrictions, we decided to run the first stage of our
 classifier on the client. The client would only send and image once it had determined that there was an face in the
 picture. This also severs to decrease the number of images being sent to and from the client. Then, the client publishes
 the cropped faces as MQTT messages to a broker server under a specific thread.

*Code for client*

***DO NOT RUN IN JUPYTER NOTEBOOK:*** The code uses openCV's imshow to display the labelled frame. This function usually crashes jupyter's server.

In [ ]:
from kivy.app import App
from kivy.uix.widget import Widget
from kivy.lang import Builder
from kivy.uix.gridlayout import GridLayout
from kivy.uix.image import Image
from kivy.clock import Clock
from kivy.properties import ListProperty
from kivy.graphics.texture import Texture
import time
from threading import Thread, Event, Lock
import cv2
from queue import Queue
import pickle
import struct
import paho.mqtt.client as mqtt
from kivy.uix.label import Label
from kivy.graphics import Color, Rectangle
from kivy.core.window import Window


Builder.load_string('''
<FormattedLabel>:
    text: root.text
    fontSize: 70
    color: (0, 0.2, .4, 1)
    size_hint: 1, 0.04
    background_color: 0,0,0,1
    '''
    )
'''
<Divider>:
color: (1, 0.2, .4, .2)
background_color: 1,0,0,0
'''
# class for managing the label display
class FormattedLabel(Label):
    background_color = ListProperty()

    def __init__(self):
        super(FormattedLabel, self).__init__()
        Clock.schedule_once(lambda dt: self.initialize_widget(), 0.002)

    def initialize_widget(self):
        self.canvas.before.add(Color(self.background_color))
        self.canvas.before.add(Rectangle(pos=self.pos, size=self.size))
        self.text_size = self.size
        self.text ="Searching for face..."
        self.halign = 'center'
        self.valign = 'top'
        self.bold = True

    def update(self, text):
        self.text = text

class Divider(Widget):
    def __init__(self, **kwargs):
        super(Divider, self).__init__(**kwargs)

        with self.canvas:
            Color(1, 0, 0, 1)  # set the colour to red
            self.rect = Rectangle(pos=self.center,
                                  size=(Window.size[0],
                                        self.height/8.))
    def update(self, color):
        with self.canvas:
            Color(color)  # set the colour to red
            self.rect = Rectangle(pos=self.top,
                                  size=(Window.size[0],
                                        self.height/8.))

#class for displaying camerafeed
class KivyCamera(Image):
    def __init__(self, capture, fps, **kwargs):
        super(KivyCamera, self).__init__(**kwargs)
        self.capture = capture
        Clock.schedule_interval(self.update, 1.0 / fps) # updates in response to framerate

    def update(self, dt):
        ret, frame = self.capture.read()
        if ret:
            buf1 = cv2.flip(frame, 0)
            buf = buf1.tostring()
            image_texture = Texture.create(
                size=(frame.shape[1], frame.shape[0]), colorfmt='bgr')
            image_texture.blit_buffer(buf, colorfmt='bgr', bufferfmt='ubyte')
            self.texture = image_texture


#main up
class RecogApp(App):

    def build(self):
        self.capture = cv2.VideoCapture(0)
        self.my_camera = KivyCamera(capture=self.capture, fps=30)
        self.total = GridLayout(rows=2)
        self.label = FormattedLabel()
       # self.divider = Divider()
        self.total.add_widget(self.my_camera)
        #self.total.add_widget(self.divider)
        self.total.add_widget(self.label)
        return self.total

    def on_stop(self):
        print("*** closing...")
        exit.set()
        cam.join()


    def on_start(self):
        QOS = 0
        BROKER = 'test.mosquitto.org'
        PORT = 1883

        def on_connect(client, userdata, rc, *extra_params):
            print('Connected with result code='+str(rc))
            client.subscribe("aiproj/facrecog/response", qos=QOS)

        def send_image(face, frame):
            (result, num) = client.publish('aiproj/facrecog/image', face, qos=QOS)
            print(result, num)
            if result != 0:
                print('PUBLISH returned error:', result)

        def on_message(client, data, msg):
            if msg.topic == "aiproj/facrecog/response":
                if msg:
                    print("recieved message: ", str(msg.payload.decode()))
                    self.label.update("Emotion: "+ str(msg.payload.decode()))

        def detectAndDisplay(frame):
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frame_gray = cv2.equalizeHist(frame_gray)
            faces = face_cascade.detectMultiScale(frame_gray)
            face_list = []
            i = 0
            for (x,y,w,h) in faces:
                center = (x + w//2, y + h//2)
                face_list.append(frame[y:y+h, x:x+w])
            return frame, face_list, faces

        def poll():
            while True:
                flag, frame = self.capture.read()
                labels = []
                if flag:
                    frame_labelled, face_list, bounds = detectAndDisplay(frame)
                    if len(face_list) == 0:
                        self.label.update("Searching for face...")
                    for i in range(len(face_list)):
                        a_face = face_list[i]
                        a_face = pickle.dumps(a_face)
                        send_image(a_face, frame)
                if exit.is_set():
                    break
            #time.sleep(.5)

        #get the dimintions of the image
        pos_frame = self.capture.get(cv2.CAP_PROP_POS_FRAMES)
        face_cascade_name = 'haarcascade_frontalface_default.xml'
        face_cascade = cv2.CascadeClassifier()
        face_cascade.load(face_cascade_name)

        '''
        server = Thread(target=start_server)
        start the thread
        server.daemon = True
        server.start()
        '''
        global client
        global exit
        global cam
        exit = Event()
        client = mqtt.Client()
        client.on_connect = on_connect
        client.on_message = on_message
        client.connect(BROKER, PORT, 60)
        cam = Thread(target=poll)
        client.loop_start()
        cam.start()

# Run client
app = RecogApp()
# app.run()

The Raspberry Pi (server) is subscribed to the client’s topic. When client publisheds a message (image), the Pi grabs
the image and puts it on a queue. The pi has a user-specified number of threads read to pull an image of queue, use the
model to predict the facial expression, then converts it to byes and sends it to a the server under a different thread.
The client will be subscribed to that thread, once it receives a message, it will display it on the screen.

*Code for the server*

In [ ]:
def run_server():
    import json
    import sys
    from threading import Thread, Event, Lock
    from queue import Queue
    import time
    import struct
    import pickle
    from tensorflow import keras
    import numpy
    import tensorflow as tf
    import paho.mqtt.client as mqtt
    import cv2 as cv

    QOS = 0
    BROKER = 'test.mosquitto.org'
    PORT = 1883

    #thread class to process data
    class myThread (Thread):
       def __init__(self, threadID, name, client):
          Thread.__init__(self)
          self.threadID = threadID
          self.name = name
          self.client =client

       def run(self):
          print ("Starting " + self.name)
          process_data(self.name, self.client, model)
          print ("Exiting " + self.name)

    #call back for mqtt client
    def on_message(client, data, msg):
        if msg.topic == "aiproj/facrecog/image":
            if msg:
                print("message")
                print("item")
                workQueue.put(msg.payload)
                print("released lock")
                print(len(workQueue))

    # call back for mqtt client
    def on_connect(client, userdata, rc, *extra_params):
       print('Connected with result code='+str(rc))
       client.subscribe("aiproj/facrecog/image", qos=QOS)

    #closes client and terminates threads
    def kill_command(threads, client):
        print("closing")
        client.close()
        exitFlag.set()
        for t in threads:
            t.join()

    # sends data to client
    def send_response(client, message):
        (result, num) = client.publish('aiproj/facrecog/response', message, qos=QOS)
        print("sent response:", message)
        if result != 0:
            print('PUBLISH returned error:', result)

    #uses model to get prediction
    def predict(data, model):
        frame = cv.resize(data, (256, 256), interpolation=cv.INTER_AREA)
        with session.graph.as_default():
            keras.backend.set_session(session)
            predictions = model.predict(numpy.reshape(frame, (1, 256, 256, 3)))
            print(predictions)
            return predictions[0].tolist()

    #converts the prediction to a string
    def getLabel(prediction):
        emotions = ["neutral", "smiling", "sad", "surprise-shock", "angry", "disgusted", "fearful"]
        response = emotions[prediction.index(max(prediction))]
        prediction.pop(prediction.index(max(prediction)))
        if max(prediction) > .5:
            response = response + "/" + emotions[prediction.index(max(prediction))]
        return response

     #work function for threads
    def process_data(threadName, client, model):
        while True:
            data = workQueue.get()
            if data:
                print (threadName, "processing an image")
                item = pickle.loads(data)
                message = getLabel(predict(item, model))
                print(threadName, "processed", message)
                #send precition
                sendLock.acquire()
                send_response(client, message)
                sendLock.release()
            else:
                print("...")
            if exitFlag.is_set():
                break
            time.sleep(.25)
    #
    def init(thread_nums):
        print("*****")
        print("Initializing connection...")
        print("*****")
        client = mqtt.Client()
        threads = []
        client.on_connect = on_connect
        client.on_message = on_message
        client.connect(BROKER, PORT, 60)
        for threadID in range(thread_nums):
            threads.append(myThread(threadID, "Thread"+str(threadID), client))
        for t in threads:
            t.start()
        return threads,client



    def load_model():
        print("Geting model files...")
        # # load json and create model
        json_file = open('model.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        print("*****")
        print("Loading models...")
        print("*****")
        session = tf.Session(graph=tf.Graph())
        with session.graph.as_default():
            keras.backend.set_session(session)
            loaded_model = keras.models.model_from_json(loaded_model_json)
            loaded_model.load_weights("model.h5")
            return loaded_model, session
        print("*****")
        print("Loaded model from disk")
        print("*****")
        # # evaluate loaded model on test data
        loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        #graph = tf.get_default_graph()
        print("*****")
        print("Model compiled")
        print("*****")


    global model
    global session
    global sendLock
    global workQueue
    model, session =load_model()
    exitFlag = Event()
    sendLock = Lock()
    connected = Event()
    workQueue = Queue(30)
    threads,client = init(3)
    try:
        client.loop_start()
    except KeyboardInterrupt:
        kill_command(threads, client)

# Run server
# run_server()

### Results
*Please watch ./demo.mov for a demonstration*

Overall, our project was a partial success. We were able to deploy our classifier on a raspberry pi and make predictions
for frames sent by a client. But the expression-recognition process was a bit slow. This was because drawing manipulating images
 on the client was very resource intensive.

Even if our model performed just as well as Sharma's model with an 85% accuracy, it did not have enough training data to understand the
nuances of facial expressions. For example, the model has a hard time differentiating between the shock and anger. This is because the datapoints
 for both expressions had people opening their mouths in them.

Additionally, our models were significantly influenced by the peculiarities of our training data. Having used phrases that associate with the emotions,
our images had similarities inherent to that phrase. (For example, if we search "syrian war" for sad, the resulting images would show the extreme forms of sadness.)
We believe this made our model make unhelpful associations between the similarities of the images and the emotion. For example, one of the phrases used to search for images
of neutral facial expression was "passport photo." This meant that the pictures had eyes looking directly as the camera. This is reflected in our observations. Frames in which a
subject is looking directly at the camera are labelled as neutral.

### Implications

There are obvious ethical issues regarding applications which capture video and transmit it elsewhere via internet. Were this
 to be a product the users purchased, there would need to be specific security and privacy assurances.

 Though we never achieved it, our original goal as to have the facial running of an AR headset as the client. This would be a prototype,
 with, in the future having a product for autistic children. One of the symptoms of autism, particularly in children is
 difficulty reading facial expressions. Such an application would run on a set of AR glasses, which are indistinguishable
 from regular glasses these days. A child with autism could have some assistance identifying facial expressions, which the child could learn as well.

Our project also shows the importance of responsible data collection and training in machine learning. Models reflect their training.
In our project, we noticed that the model made unintended associations between some facial features and some facial
expressions based on its training data. In "real" applications, ignoring bias in the training sets can result in models that make inaccurate predictions
that influence peoples lives.

### Conclusion
Facial recognition is growing sub-field in Object-detection research. Our project focused on recognizing facial expressions and
classifying their facial expressions. We used openCV's cascading classifier models to find faces in images and trained a CNN
to classify the facial expressions. These two tasks were distributed between a server and a client that communicated using
the MQTT messaging protocol. Even if our project was not a complete success, it still demonstrated the potential of facial recognition
systems. They can be used as aids to people who need assistance in understanding emotions. They are also versatile. The detection task can be
distributed to between multiple devices in a pipeline. With a fast enough messaging system, this division of labor lets a network
of simple machines--like the raspberry pi--conduct resource intensive recognition tasks.

### Citations

[Sharma, Gaurav. Real Time Facial Expression Recognition. Real Time Facial Expression Recognition. Medium, n.d.](https://medium.com/datadriveninvestor/real-time-facial-expression-recognition-f860dacfeb6a.)

[Zenodo. The Japanese Female Facial Expression Database.](https://zenodo.org/record/3451524#.Xs5tLPJKiV4)

[VISGRAF. FaceDB.](http://app.visgraf.impa.br/database/faces/)

[Bosler, Fabian. Image Scraping with Python. Medium. September, 2019.](https://towardsdatascience.com/image-scraping-with-python-a96feda8af2d)